# 1. import libraries

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as v_utils
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from collections import OrderedDict
from skimage import io
from skimage.transform import resize
import os

# 2. Data load

In [2]:
file_dir = 'C:\\Users\\NHNEnt\\Downloads\\thumbnails_features_deduped_sample\\thumbnails_features_deduped_sample\\*'

import glob
filelist = glob.glob(file_dir)

list_imagedir = np.array([glob.glob(imagepath+'\\*.jpg') for imagepath in filelist])

image = np.array([np.array(io.imread(fname = imagepath)) for imagedir in list_imagedir for imagepath in imagedir])



# 2.1 data preprocessing

In [3]:
def fresize(image,size):
    result = resize(image,(size,size),mode='reflect')
    return result
image_pre = np.array([fresize(img,64) for img in image])

# 3. Model

In [23]:
def deconv(ch_in,ch_out,k_size=4,stride=2,pad=1,bn=True):
    layers = []
    layers.append(nn.ConvTranspose2d(ch_in,ch_out,k_size,stride,padding=pad))
    if bn:
        layers.append(nn.BatchNorm2d(ch_out))
    return nn.Sequential(*layers)

def conv(ch_in, ch_out, k_size, stride, pad=1, bn=True):
    layers = []
    layers.append(nn.Conv2d(ch_in,ch_out,k_size,stride,padding=pad))
    if bn:
        layers.append(nn.BatchNorm2d(ch_out))
    return nn.Sequential(*layers)
    
def normal_init(m,mean,std):
    if isinstance(m,nn.ConvTranspose2d) or isinstance(m,nn.Conv2d):
        m.weight.data.normal_(mean,std)
        b.bias.data.zero_()
        
def bn_init(m):
    if isinstance(m,nn.BatchNorm2d):
        m.weight.data.fill(1)
        m.bias.data.zero_()

In [65]:
class G(nn.Module):
    def __init__(self,dim_z=100,input_size=64,out_dim=64,colch=3,leaky=0.2):
        super(G,self).__init__()
        
        self.dim_z = dim_z
        self.input_size = input_size
        self.out_dim = out_dim
        self.colch = colch
        self.leaky = leaky
        
        self.fc = deconv(dim_z,out_dim*8,int(input_size/16),1,0,bn=False)
        self.deconv1 = deconv(out_dim*8,out_dim*4,4)
        self.deconv2 = deconv(out_dim*4,out_dim*2,4)
        self.deconv3 = deconv(out_dim*2,out_dim,4)
        self.deconv4 = deconv(out_dim,colch,4,bn=False)
    
        self.weight_init()
        
    def weight_init(self,mean=0.,std=0.02):
        for m in self._modules:
            normal_init(m,mean,std)
            bn_init(m)
    

        
    def forward(self,z):
            z = z.view(z.size(0),z.size(1),1,1)
            out = self.fc(z)
            out = F.leaky_relu(self.deconv1(out),self.leaky )
            out = F.leaky_relu(self.deconv2(out),self.leaky )
            out = F.leaky_relu(self.deconv3(out),self.leaky )
            out = F.tanh(self.deconv4(out))
            return out
        
class D(nn.Module):
    def __init__(self,input_size=64,conv_dim=64, colch=3):
        super(D,self).__init__()
        
        self.input_size = input_size
        self.colch = colch
        self.conv_dim = conv_dim
        
        self.conv1 = conv(input_size,conv_dim,4,2,bn=False)
        self.conv2 = conv(conv_dim,conv_dim*2,4,2)
        self.conv3 = conv(conv_dim*2,conv_dim*4,4,2)
        self.conv4 = conv(conv_dim*4,conv_dim*8,4,2)
        self.fc = conv(conv_dim*8,1,4,1,0,bn=False)
        
        self.weight_init
    def weight_init(self,mean=0.,std=0.02):
        for m in self._modules:
            normal_init(m,mean,std)
            bn_init(m)
            
    def forward(self,x):
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))
        out = F.relu(self.conv4(out))
        out = self.fc(out).squeeze()
        return out
        
        

In [70]:
class DCGAN(object):
    def __init__(self,input_size=64,dim_z=100,colch=3,leaky=0.2,
                end_g_dim=64,start_d_dim=64):
        self.input_size= input_size
        self.dim_z = dim_z
        self.colch = colch
        self.leaky = leaky
        self.end_g_dim = end_g_dim
        self.start_d_dim = start_d_dim
        
        self.G = None
        self.D = None
        self.G_optim = None
        self.D_optim = None
        
        self.build_model()
        
    def build_model(self):
        self.G = G(self.dim_z,self.input_size,self.end_g_dim,self.colch,self.leaky)
        self.D = D(self.input_size,self.start_d_dim, self.colch)


    def train(self, data, fixed_z, batch_size=128,
              epoch=10,lr=0.002,check_step=100):
            
        self.G_optim = torch.optim.Adam(self.G.parameters(),lr=lr)
        self.D_optim = torch.optim.Adam(self.D.parameters(),lr=lr)
        
        
        loss_func = nn.BCELoss()
        
        data = Variable(torch.from_numpy(data))
        
        counter = 0
        
        if not os.path.isdir("./dcgan_result"):
            os.mkdir("./dcgan_result")

        for e in range(epoch):
            for i in range(int(len(data)/batch_size)):
                
                # update D
                z = Variable(torch.rand(batch_size,self.dim_z))
                
                self.D_optim.zero_grad()
                
                G_fake = self.G.forward(z)
                D_fake = self.D.forward(G_fake)
                D_real = self.D.forward(data[i*batch_size:(i+1)*batch_size])
                
                D_loss = (torch.sum(loss_func(D_fake,Varaible(torch.zeros(batch_siz,1))))+
                          torch.sum(loss_func(D_real,Varaible(torch.ones(batch_size,1)))))

                D_loss.backward(retain_variables=False)
                
                self.D_optim.step()
                
                # update G
                z = Variable(torch.rand(batch_size,self.dim_z))
                
                self.G_optim.zero_grad()
                
                G_fake = self.G.forward(z)
                D_fake = self.D.forward(G_fake)
                
                G_loss = torch.sum(loss_func(D_fake,Variable(torch.ones(batch_size,1))))
                
                G_loss.backward()
                
                self.G_optim.step()
                
                if counter % check_step == 0:
                    print("Epoch [%d/%d], Step [%d/%d], D_loss : %.4f, G_loss : %.4f"%(
                        e,epoch,i,(len(data)/batch_size),D_loss,G_loss))
                    
                
                
                
                

# 4.train

In [71]:
fixed_z = Variable(torch.rand(30,100))

In [72]:
dcgan = DCGAN()

In [73]:
dcgan.train(image_pre,fixed_z)

a


RuntimeError: Need input of dimension 4 and input.size[1] == 64 but got input to be of shape: [128 x 3 x 64 x 64] at d:\projects\pytorch\torch\lib\thnn\generic/SpatialConvolutionMM.c:47

In [74]:
image_pre.shape

(2174, 64, 64, 3)